In [1]:
import numpy as np
from scipy.optimize import linprog
import pandas as pd
from itertools import combinations
from qpsolvers import solve_qp
from scipy.sparse import csc_matrix

In [2]:
# Decision variables: [A1, B1, C1, D1, A2, B2, C2, D2, A3, B3, C3, D3, A4, B4, C4, D4, A5, B5, C5, D5, A6, B6, C6, D6]
# Objective function coefficients: minimize pairwise distances of points with same line number

letters = ['A', 'B', 'C', 'D']
n_letters = len(letters)
n_questions = 10

n_differences = n_questions * n_letters 
n_variables = n_differences + n_questions * n_letters

# Objective: minimize sum of all differences
c = np.array([0] * (n_questions * n_letters) + [1] * n_differences)

# Inequality constraints (Ax <= b)
A_ub = np.array([])
b_ub = np.array([])
targets = np.random.random((n_questions, n_letters))
for i in range(n_questions):
    for j in range(n_letters):
        cons = np.zeros(n_variables)
        cons[i * n_letters + j] = 1
        cons[n_letters * n_questions + i * n_letters + j] = -1
        A_ub = np.vstack((A_ub, cons)) if A_ub.size else cons
        b_ub = np.append(b_ub, targets[i, j])
        
        cons = np.zeros(n_variables)
        cons[i * n_letters + j] = -1
        cons[n_letters * n_questions + i * n_letters + j] = -1
        A_ub = np.vstack((A_ub, cons))
        b_ub = np.append(b_ub, -targets[i, j])
    
b_ub = np.zeros(len(A_ub))
# Equality constraints (Ax = b)
A_eq = np.array([])
for i in range(n_questions):
    cons = np.zeros(n_variables)
    indexes = [i * n_letters + j for j in range(n_letters)]
    cons[indexes] = 1
    A_eq = np.vstack((A_eq, cons)) if A_eq.size else cons
    
b_eq = np.ones(len(A_eq))

# Bounds
bounds = [(0, 1) for _ in range(n_letters * n_questions)] + [(-1, 1) for _ in range(n_differences)]

In [3]:
# Add new constraints
def add_answer(A_eq, b_eq, answer, answer_score):
    answer_indexes = [i * n_letters + letters.index(a) for i, a in enumerate(answer)]
    cons = np.zeros(n_variables)
    cons[answer_indexes] = 1
    A_eq = np.vstack((A_eq, cons))
    b_eq = np.append(b_eq, answer_score)
    return A_eq, b_eq


answer = ['A'] * n_questions
answer_score = 4
A_eq, b_eq = add_answer(A_eq, b_eq, answer, answer_score)

answer = ['B'] * n_questions
answer_score = 2
A_eq, b_eq = add_answer(A_eq, b_eq, answer, answer_score)

answer = ['C'] * n_questions
answer_score = 1
A_eq, b_eq = add_answer(A_eq, b_eq, answer, answer_score)

answer = ['D'] * n_questions
answer_score = 3
A_eq, b_eq = add_answer(A_eq, b_eq, answer, answer_score)

answer = ['A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B']
answer_score = 8
A_eq, b_eq = add_answer(A_eq, b_eq, answer, answer_score)

In [4]:
# Solve
P = csc_matrix(np.zeros((n_variables, n_variables)))
q = c
G = csc_matrix(A_ub)
h = b_ub
A = csc_matrix(A_eq)
b = b_eq
lb = np.array([elem[0] for elem in bounds])
ub = np.array([elem[1] for elem in bounds])
x = solve_qp(P, q, G, h, A, b, lb=lb, ub=ub, solver='osqp')

In [5]:
res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, integrality=1)
x_lineprog = res.x

In [6]:
# Show results
data = x[:(n_letters * n_questions)].reshape(n_questions, n_letters)
df = pd.DataFrame(data, columns=['A', 'B', 'C', 'D'])
df.index += 1
df.index.name = 'Question'
df.round(2)

,A,B,C,D
Question,,,,
1,1.00,-0.00,0.00,-0.00
2,0.12,0.68,0.01,0.19
3,0.31,-0.01,0.48,0.21
4,-0.00,-0.00,0.00,1.00
5,1.00,-0.00,0.00,-0.00
6,0.12,0.68,0.01,0.19
7,0.31,-0.01,0.48,0.21
8,-0.00,-0.00,0.00,1.00
9,1.00,-0.00,0.00,-0.00


In [7]:
# Show results
data = x_lineprog[:(n_letters * n_questions)].reshape(n_questions, n_letters)
df = pd.DataFrame(data, columns=['A', 'B', 'C', 'D'])
df.index += 1
df.index.name = 'Question'
df.round(2)

,A,B,C,D
Question,,,,
1,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0
5,1.0,0.0,0.0,0.0
6,0.0,1.0,0.0,0.0
7,-0.0,0.0,1.0,0.0
8,0.0,0.0,0.0,1.0
9,1.0,0.0,0.0,0.0
